In [42]:
import re
import sys
import numpy as np
from random import random
from math import log
from collections import defaultdict

In [43]:

infile_en = 'data/training.en'
infile_de = 'data/training.de'
infile_es = 'data/training.es'
infile_mo = 'data/model-br.en'
infile_test = 'data/test'

In [58]:
# to generate a set which consists the courpus
option =  set(chr(i) for i in range(97,123))
option =  option|{" ",".","#","0"}

full = set()
for i in option:
    for j in option:
        full = full|{i+j}
full = full -{"##"}

In [59]:
def preprocess_line(line):
        alpha = set(chr(i) for i in range(97,123))|set(chr(i) for i in range(65,91))
        res = ""
        for i in line.lower().rstrip("\n"): #there are "\n" in every end of the sentence
            if i in alpha:
                res += i
            elif i.isspace():
                res += i
            elif i.isdigit():
                res += "0"
            elif i == ".":
                res += i
        return res


In [60]:
def generate_LM(infile, full):    
    bi_counts = defaultdict(list)
    hi_counts = defaultdict(list)
    with open(infile) as f:
        for line in f:
            #print(line)
            line = preprocess_line(line)
            line = "#"+line+"#"    #add the marks of beginning and end 
            #print(line)
            for j in range(len(line)-(1)):
                bigram = line[j:j+2]
                hi = bigram[:1]
                try:
                    hi_counts[hi][0] += 1
                except:
                    hi_counts[hi].append(1)
                try:
                    bi_counts[bigram][0] += 1
                except:
                    bi_counts[bigram].append(1.005)
                    hi_counts[hi][0] += 0.005
                    bi_counts[bigram].append(hi)
                    hi_counts[hi].append(bigram)
            
                full = full- {bigram}
    #start to do the process of smooth
    for bigram in full:
        hi = bigram[:1]
        try:
            hi_counts[hi][0] += 0.005
        except:
            hi_counts[hi].append(0.005)
        
        bi_counts[bigram].append(0.005)
        bi_counts[bigram].append(hi)
        hi_counts[hi].append(bigram)
    """
    we generate two dict in collaboration to restore the counts and distribution of our language model
    
    Dict1: bi_counts = {"bigram" : [counts_in_text, two_c_hist, condition_probability]} 
    Dict2: hi_counts = {"c_hist" : [counts_in_trigram, gram_in_corpus_begin_with_his]} 
    
    """
    for s in sorted(bi_counts.keys()):
        bi_counts[s].append(bi_counts[s][0]/hi_counts[bi_counts[s][1]][0])
    return bi_counts,hi_counts

In [61]:

bi_en_counts,hi_en_counts = generate_LM(infile_en, full)
bi_de_counts,hi_de_counts = generate_LM(infile_de, full)
bi_es_counts,hi_es_counts = generate_LM(infile_es, full)


In [62]:

def generate_from_LM(bi_counts, hi_counts):
    key = "#"
    para = key
    i = 0
    while i < 300:
        opt = list(hi_counts[key][1:])
        prob = list(bi_counts[key][2] for key in opt)
        bins = np.cumsum(prob)
        choose = np.digitize(np.random.rand(1), bins)
        key = opt[choose[0]]
        i +=1
        if key[1] == "#":
            para += "\n"
            para += "#"
            i -= 1    
        para += key[1]
        key = key[1:]
    res =""
    for c in para:
        if c != "#":
            res += c
    return res

In [63]:
para_1 = generate_from_LM(bi_en_counts, hi_en_counts)
print(para_1)

amund dofay othannciand o tstreseeroll th metuscan qur ic oumakncon therl perithins aindalathikefurony anotouirund s trthe t is ges raty ctisecommaton omatiom an io opon iril dmpres tirncoffeeeemrawon ericucotucathens atount thoonivayenstshem gres nenthe hale ideralartlver ounay eldsonino te in oun 


In [64]:
def calculate_perp(infile_test, bi_counts):
    logsum = 0
    with open(infile_test) as f:
        sum_t = 0
        for line in f:
            line = preprocess_line(line) 
            line = "#"+line+"#"
            sum_t += (len(line)-1)
            for i in range(len(line)-(1)):
                bi = line[i:i+2]
                logsum += log(bi_counts[bi][2],2)
           
    N = sum_t
    print(N)
    #print(prob)
    perp = pow(2,-1/N * logsum)
    print(perp)
    return perp


In [66]:
calculate_perp(infile_test, bi_en_counts)
calculate_perp(infile_test, bi_de_counts)
calculate_perp(infile_test, bi_es_counts)

13553
11.475353772208646
13553
21.9541154698943
13553
27.289234432363397


27.289234432363397